# Подключение необходимых для работы библиотек

In [23]:
import numpy as np
import scipy as sp
from scipy.linalg import lu
import copy

# Выражение для исходной системы AX = f

In [24]:
A = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
             [1, -2, 1, 0, 0, 0, 0, 0, 0],
             [0, 1, -2, 1, 0, 0, 0, 0, 0],
             [0, 0, 1, -2, 1, 0, 0, 0, 0],
             [0, 0, 0, 1, -2, 1, 0, 0, 0],
             [0, 0, 0, 0, 1, -2, 1, 0, 0],
             [0, 0, 0, 0, 0, 1, -2, 1, 0],
             [0, 0, 0, 0, 0, 0, 1, -2, 1],
             [1, 2, 2, 2, 2, 2, 2, 2, 1]])

f = np.array([1, 2 / 4, 2 / 9, 2 / 16, 2 / 25, 2 / 36, 2 / 49, 2 / 64, -3])


# Вспомогательные функции

In [25]:
def norm(x):
    abs_arr = np.absolute(x)
    return np.amax(abs_arr)

# Метод Зейделя

In [26]:
def seidel_method(A, f, precision):
    
    D = np.diag(np.diag(A))
    U = np.triu(A, 1)
    L = np.tril(A, -1)
    
    inverted = np.linalg.inv(D + L)
    
    B = -1 * inverted.dot(U)
    print(inverted)
    print(f)
    F = inverted.dot(f)
    
    n = f.size
    x = np.zeros(n)
    
    iters = 0
    
    error = norm(A.dot(x) - f)
    while (error > precision):
        x = B.dot(x) + F
        error = norm(A.dot(x) - f)
        
        print('Невязка на {0} итерации: '.format(iters))
        print(error)
        
        print('Решение на {0} итерации: '.format(iters))
        print(x)
        print('\n\n\n')
        
        iters += 1
        
    return x, error


# Метод Гаусса

In [29]:
def element_swap(A):
    n = A[0].size
    max_elem = 0
    index = 0
    
    for i in range(n):
        if (A[i][0] > max_elem):
            max_elem = A[i][0]
            index = i
                
    A[[0, index]] = A[[index, 0]]
    return A

test_A = np.array([[4, 1, 1],
                  [3, 5, 1],
                  [1, 1, 3]], dtype=float)

test_F = np.array([3, 7, 4], dtype=float)


def gauss_direct(A, f):
    A = element_swap(A)
    n = A[0].size
    if (n == 1):
        return
    
    for i in range(1, n):
        factor = A[i, 0] / A[0, 0]
        A[i] = A[i] - factor * A[0]
        f[i] = f[i] - factor * f[0]
        
    gauss_direct(A[1:, 1:], f[1:])
    
    return A, f

def gauss_reverse(A, f):
    
    n = A[0].size
    if (n == 1):
        return
    
    for i in reversed(range(0, n - 1)):
        factor = A[i, n - 1] / A[n - 1, n - 1]
        A[i] = A[i] - factor * A[n - 1]
        f[i] = f[i] - factor * f[n - 1]
    
    gauss_reverse(A[:(n - 1), :(n - 1)], f[:(n - 1)])
    
    ans = np.zeros(n)
    for i in range(n):
        ans[i] = f[i] / A[i][i]
    
    return ans

# Непосредственно программа с тестированием методов

In [30]:
res, error = seidel_method(test_A.copy(), test_F.copy(), 10e-14)
print('Итоговое решение по Зейделю: ')
print(res)
print('Итоговая невязка: ')
print(error)
print('\n\n\n')

resA, resf = gauss_direct(test_A.copy(), test_F.copy())
res = gauss_reverse(resA.copy(), resf.copy())
print('Итоговое решение по Гауссу:')
print(res)
print('\n\n\n')



print('Правильное решение через питоновские алгоритмы (для проверки): ')
res = np.linalg.solve(test_A.copy(), test_F.copy())
print(res)
print('\n\n\n')



[[ 0.25        0.          0.        ]
 [-0.15        0.2         0.        ]
 [-0.03333333 -0.06666667  0.33333333]]
[3. 7. 4.]
Невязка на 0 итерации: 
1.7166666666666668
Решение на 0 итерации: 
[0.75       0.95       0.76666667]




Невязка на 1 итерации: 
0.2124999999999999
Решение на 1 итерации: 
[0.32083333 1.05416667 0.875     ]




Невязка на 2 итерации: 
0.024513888888888946
Решение на 2 итерации: 
[0.26770833 1.064375   0.88930556]




Невязка на 3 итерации: 
0.0025868055555555713
Решение на 3 итерации: 
[0.26157986 1.06519097 0.89107639]




Невязка на 4 итерации: 
0.0002381365740742325
Решение на 4 итерации: 
[0.26093316 1.06522483 0.89128067]




Невязка на 5 итерации: 
2.1556712963999303e-05
Решение на 5 итерации: 
[0.26087363 1.06521969 0.89130223]




Невязка на 6 итерации: 
1.984471451166314e-06
Решение на 6 итерации: 
[0.26086952 1.06521784 0.89130421]




Невязка на 7 итерации: 
2.396195024623182e-07
Решение на 7 итерации: 
[0.26086949 1.06521747 0.89130435]




Невяз

# Подсчет корней характеристического уравнения и числа обусловленности

In [7]:
lambdas, eigenvectors = np.linalg.eig(A)
print(lambdas)

lambda_max = lambdas.max()
lambda_min = lambdas.min()

print('Максимальный корень ХУ: ')
print(lambda_max)
print('Минимальный корень ХУ: ')
print(lambda_min)

print('\n\n\n')

norm_a = norm(A)
norm_a_inversed = norm(np.linalg.inv(A))

print(norm_a)
print(norm_a_inversed)

print('Число обусловленности: ')
print(norm_a * norm_a_inversed)

[ 1.79620635 -3.8571539  -3.41421356 -2.86417214 -2.         -1.55703634
 -0.51784397 -0.58578644  1.        ]
Максимальный корень ХУ: 
1.7962063457382969
Минимальный корень ХУ: 
-3.8571538967671044




2
1.9999999999999996
Число обусловленности: 
3.999999999999999
